In [ ]:
import cv2
from keras.preprocessing import image
import numpy as np
import pathlib
from skimage.transform import resize
import os
import re
import pickle
from keras.utils.np_utils import to_categorical

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input as ResNet_Pre
from tensorflow.keras.applications.densenet import preprocess_input as DenseNet_Pre
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as MobileNet_Pre

In [ ]:
def vidToImages(videoFile, dim=(224,224), skipValue=2):
    
    cap = cv2.VideoCapture(videoFile)

    ImgArr = []

    while(cap.isOpened()):
        frameId = cap.get(cv2.CAP_PROP_POS_FRAMES)
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % skipValue == 0):
            resized = resize(frame, preserve_range=True, output_shape=dim).astype(int)
            ImgArr.append(resized)
    cap.release()
    
    return np.array(ImgArr)

In [ ]:
def preprocess(Arr, process_func):
    img_array = []
    for img in Arr:
        t = image.img_to_array(img)
        t = np.expand_dims(t, axis=0)
        t = process_func(t)
        img_array.append(t)
    
    return np.vstack(img_array)

In [ ]:
model_ResNet = ResNet50(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_ResNet.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [ ]:
model_DenseNet = DenseNet121(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_DenseNet.summary()

29097984/29084464 [==============================] - 0s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_2[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

In [ ]:
model_MobileNet = MobileNetV2(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_MobileNet.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                

In [ ]:
MainFolder = 'ucf_action'
dim = (224,224)
skipValue = 2
model = model_MobileNet
preProcess_func = MobileNet_Pre
filename_X = 'X_MobileNetV2.p'
filename_Y = 'Y_MobileNetV2.p'


classes = {'Diving-Side':0,
           'Golf-Swing-Back':1,
           'Golf-Swing-Front':1,
           'Golf-Swing-Side':1,
           'Kicking-Front':2,
           'Kicking-Side':2,
           'Lifting':3,
           'Riding-Horse':4,
           'Run-Side':5,
           'SkateBoarding-Front':6,
           'Swing-Bench':7,
           'Swing-SideAngle':8,
           'Walk-Front':9}

total_classes = 10


X = []
Y = []


for path, dirs, files in os.walk(MainFolder):
    
    if re.search(r'^ucf_action\\.*\\.*\\gt.*', path) is not None:
        continue
    
        flag = 0
        
        for file in files:
            ext = pathlib.Path(file).suffix
            if ext == '.avi':
                flag = 1
                actual_path = os.path.join(path, file)
                t = vidToImages(actual_path, dim=dim, skipValue=skipValue)
                t = preprocess(t, preProcess_func)
                t = model.predict(t)
                X.append(t)
                Y.append(classes[p])
                break
        
        if flag == 0:
            i = -1
            for img_name in files:
                
                file_extension = pathlib.Path(img_name).suffix
                if file_extension == '.jpg':
                    i = i + 1
                    if i % skipValue == 0:
                        actual_path = os.path.join(path, img_name)
                        img = image.load_img(actual_path, target_size=dim)
                        img = image.img_to_array(img)
                        # use skip value
                        img = np.expand_dims(img, axis=0)
                        img = preProcess_func(img)
                        stack.append(img)

            if len(stack) != 0:
                X.append(model.predict(np.vstack(stack)))
                Y.append(classes[p])


Y = to_categorical(Y, num_classes=total_classes)

print("Saving to a pickle file...")
with open(filename_X, 'wb') as fp:
    pickle.dump(X, fp)

with open(filename_Y, 'wb') as fp:
    pickle.dump(Y, fp)
    
print("Done.")

Saving to a pickle file...
Done.
